In [139]:
pip install fasttext

In [140]:
import fasttext
import pandas as pd
import nltk
import emoji
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score, precision_score, recall_score

# Reading and Preprocessing the Data

In [141]:
data = pd.read_csv("../../Danish/Danish/offenseval-da-training-v1.tsv", sep='\t')[:-1]

texts = data["tweet"].apply(str)
labels = data["subtask_a"].apply(str)
#print(data.loc[2883])
texts

0       Jeg tror det vil være dejlig køligt, men jeg v...
1       Så kommer de nok til at investere i en ny cyke...
2       Nu er det jo også de Ikea-aber der har lavet s...
3       128 Varme emails, er vi enige om at det er sex...
4       Desværre tyder det på, at amerikanerne er helt...
                              ...                        
2955    Har sgu lidt en anelse om... det her kunne mås...
2956    Ind og ruske tremmer med hende,Den syge kælling!!
2957                                             fedtmule
2958                                    ##HAR I HØRT DET?
2959    Kommer det bag på nogen? Det er jo fucking var...
Name: tweet, Length: 2960, dtype: object

In [142]:
#tokenization and replacing emojis
texts_tokenized = []
for el in texts:
    texts_tokenized.append([emoji.demojize(el).lower() for el in el.split()]) #nltk.word_tokenize(el)

    
texts_tokenized[2883]

['@user',
 'næste',
 'gang',
 'pastaen',
 'er',
 'brændt',
 'på',
 '!',
 ':face_with_tears_of_joy:',
 'det',
 'jo',
 'sygt...']

In [143]:
#normalization
texts_normalized = []
for sent in texts_tokenized:
    
    # remove emojis
    #sent = [el for el in sent if not bool(re.search(r":.*:", el))] #or replace with just name?
    
    # remove repeating characters
    sent = [re.sub(r"(.)\1{2,}", r"\1", el) for el in sent]
    
    # remove emojis
    #sent = [re.sub(r"[!?\(\)\\&.,:><_#\[\]/]+", "", el)for el in sent]
    
    texts_normalized.append([el for el in sent if el != ""])
        
print(texts_normalized[2883])

['@user', 'næste', 'gang', 'pastaen', 'er', 'brændt', 'på', '!', ':face_with_tears_of_joy:', 'det', 'jo', 'sygt.']


In [144]:
# add the preprocessed texts back into the dataframe
texts_normalized2 = [" ".join(el) for el in texts_normalized]
data["tweet"] = texts_normalized2
data

,id,tweet,subtask_a
0,3131,"jeg tror det vil være dejlig køligt, men jeg v...",NOT
1,711,så kommer de nok til at investere i en ny cyke...,NOT
2,2500,nu er det jo også de ikea-aber der har lavet s...,OFF
3,2678,"128 varme emails, er vi enige om at det er sex...",NOT
4,784,"desværre tyder det på, at amerikanerne er helt...",NOT
...,...,...,...
2955,170,har sgu lidt en anelse om. det her kunne måske...,NOT
2956,1226,"ind og ruske tremmer med hende,den syge kælling!!",OFF
2957,2596,fedtmule,NOT
2958,1802,##har i hørt det?,NOT


In [145]:
texts = data["tweet"].apply(str)
texts, labels

(0       jeg tror det vil være dejlig køligt, men jeg v...
 1       så kommer de nok til at investere i en ny cyke...
 2       nu er det jo også de ikea-aber der har lavet s...
 3       128 varme emails, er vi enige om at det er sex...
 4       desværre tyder det på, at amerikanerne er helt...
                               ...                        
 2955    har sgu lidt en anelse om. det her kunne måske...
 2956    ind og ruske tremmer med hende,den syge kælling!!
 2957                                             fedtmule
 2958                                    ##har i hørt det?
 2959    kommer det bag på nogen? det er jo fucking var...
 Name: tweet, Length: 2960, dtype: object,
 0       NOT
 1       NOT
 2       OFF
 3       NOT
 4       NOT
        ... 
 2955    NOT
 2956    OFF
 2957    NOT
 2958    NOT
 2959    OFF
 Name: subtask_a, Length: 2960, dtype: object)

In [146]:
texts_train, texts_test, labels_train, labels_test = train_test_split(texts, labels, test_size=0.25, random_state=123)
#texts_train, texts_test, labels_train, labels_test

In [147]:
# write train and test data into files. label and tweet are separated by \t
train_file = "../../Danish/Model/train.txt"
test_file = "../../Danish/Model/test.txt"
with open(train_file, "w", encoding="utf8") as f:
    for label, tweet in zip(labels_train, texts_train):
        f.write(f"__label__{label}\t{tweet}\n")
        
with open(test_file, "w", encoding="utf8") as f:
    for label, tweet in zip(labels_test, texts_test):
        f.write(f"__label__{label}\t{tweet}\n")

## Train some models

In [148]:
# train different models and add them to a list
models = []
model1 = fasttext.train_supervised(input=train_file)
models.append(model1)

In [149]:
model2 = fasttext.train_supervised(input=train_file, epoch=25)
models.append(model2)

In [150]:
model3 = fasttext.train_supervised(input=train_file, epoch=25, lr=0.3)
models.append(model3)

In [151]:
model4 = fasttext.train_supervised(input=train_file, epoch=100, lr=0.1)
models.append(model4)

In [152]:
# use all models to predict the test file
predictions = []
for model in models:
    with open(test_file, "r", encoding="utf8") as f:
        pred_labels = []
        true_labels = []
        for line in f.readlines():
            l, t = line.split("\t")
            true_labels.append(l)
            pred = model.predict(t.strip("\n"))
            pred_labels.append(pred[0][0])
    predictions.append((pred_labels, true_labels))
len(predictions)

4

## Evaluation

In [153]:
def evaluation_metrics(pred_label,true_label):
    '''
    Returns accuracy and f1 score metrics for evaluation
    '''
    accuracy=accuracy_score(true_label,pred_label)
    f1score=f1_score(true_label,pred_label,average='macro')
    precision= precision_score(true_label, pred_label,average='macro')
    recall=recall_score(true_label, pred_label,average='macro')
    
    return (accuracy,precision,recall,f1score)

In [162]:
# evaluate
i = 1
maxf = 0
for el in predictions:
    metrics = evaluation_metrics(el[0], el[1])
    if metrics[-1] >= maxf:
        maxf = metrics[-1]
        best_model = models[i-1]
    print(f"Model{i}:\n\tAccuracy: {metrics[0]}\n\tPrecision: {metrics[1]}\n\tRecall: {metrics[2]}\n\tF1score: {metrics[3]}")
    i+=1
print(f"Best Model: {best_model}: {maxf}")

Model1:
	Accuracy: 0.9121621621621622
	Precision: 0.9557220708446866
	Recall: 0.5422535211267605
	F1score: 0.5547574307374735
Model2:
	Accuracy: 0.9202702702702703
	Precision: 0.8213282504012841
	Recall: 0.634865997178888
	F1score: 0.6806588696523527
Model3:
	Accuracy: 0.9202702702702703
	Precision: 0.8147887323943661
	Recall: 0.6411608665445588
	F1score: 0.686528478808722
Model4:
	Accuracy: 0.9081081081081082
	Precision: 0.7339971550497866
	Recall: 0.6281395397797849
	F1score: 0.6604038440773135
Best Model: <fasttext.FastText._FastText object at 0x000001F12A3142B0>: 0.686528478808722


In [155]:
for model in models:
    print(model.predict("hvad med i stedet for at anholde ham så give ham et lift til hotellet i stedet for de er skøre de svenskere"))

(('__label__NOT',), array([0.90585428]))
(('__label__OFF',), array([0.88944393]))
(('__label__OFF',), array([0.99604893]))
(('__label__OFF',), array([0.98114443]))


In [161]:
# test the best model on random samples
import random
with open(test_file, "r", encoding="utf8") as f:
    for el in random.sample(f.readlines(), 15):
        l, t = el.split("\t")
        true = l[-3:]
        pred = best_model.predict(t.strip("\n"))[0][0][-3:]
        print(f"True: {true}, Pred: {pred}\n\t{t}")

True: NOT, Pred: NOT
	jeg mener billedet kommer fra en kampagne af ddrjake. han streamer stadig på twitch og arbejder faktisk for paradox interactive, der har lavet spillet. den eneste skavank er at paradox er et svensk firma. fy da føj da.

True: NOT, Pred: NOT
	nogen der ved hvor man kan streame/downloade/købe filmen digitalt?

True: NOT, Pred: NOT
	send dem hjem og luk ikke flere ind

True: NOT, Pred: NOT
	jeg var studerende på danmark fra usa i 1994 og kan stadig snakke noget dansk. jeg elsker danmark!

True: NOT, Pred: NOT
	fremragende tilbud hvor der både er lidt til ham og hende!

True: NOT, Pred: NOT
	jeg ville ønske, migmiger brugte grammatisk komma :( "det er der, du tager fejl, drengo" punktummet til slut er ikke så vigtigt.

True: NOT, Pred: NOT
	inte ens i danmark vill man dricka dansk glögg.

True: NOT, Pred: NOT
	godt arbejde, op!

True: NOT, Pred: NOT
	jeg læste med det samme det første billede som "pludder".

True: NOT, Pred: NOT
	gg

True: NOT, Pred: NOT
	ruller med k